In [193]:
#neural network
import numpy as np
import pandas as pd


In [194]:
#load data
df = pd.read_csv(r'C:\Users\piyus\DiabetesPrediction\data\diabetes.csv')
x = df.drop('Outcome', axis=1)  # All columns except the target
y = df['Outcome']              # Only the target column
x = x.to_numpy()
y = y.to_numpy()
x=x.T

In [195]:
def scale(X):
    X_mean = np.mean(X, axis=0)
    X_std = np.std(X, axis=0)
    X_scaled = (X - X_mean) / X_std
    return X_scaled

In [196]:
x=scale(x)

In [197]:
def train_test_split(x,y,ratio,m):
    #shuffle data
    indices=np.arange(m)
    np.random.seed(42)
    np.random.shuffle(indices)

    x_shuffled= x[indices]
    y_shuffled =y[indices]

    #split data
    train_size= int(ratio*m)
    x_train,x_test=x_shuffled[:train_size] , x_shuffled[train_size:]
    y_train,y_test=y_shuffled[:train_size] , y_shuffled[train_size:]

    return x_train, x_test, y_train, y_test

In [198]:
n=x.shape[0]
m=x.shape[1]


In [199]:
x_train,x_test,y_train,y_test=train_test_split(x.T,y.T,0.8,m)

In [200]:
x_train,x_test,y_train,y_test=x_train.T,x_test.T,y_train.T,y_test.T

In [201]:
def relu(x):
        return np.maximum(0,x)

def sigmoid(z):
        z = np.clip(z, -500, 500)  # or -100 to 100
        y=1/(1+np.exp(-z))
        return y
activation=[relu,sigmoid]

def reluder(z):
        return (z > 0).astype(float)
def sigder(z):
        A = activation[1](z)
        return A * (1 - A)
derivatives=[reluder,sigder]

In [202]:
def forward(w,x,b,activation):
    z=np.dot(w,x)+b
    a=activation(z)
    return a,z


In [203]:

def compute_cost(y, y_hat):
    m = y.shape[0]
    epsilon = 1e-15  # To avoid log(0)
    y_hat = np.clip(y_hat, epsilon, 1 - epsilon)
    cost = - (1/m) * np.sum(
        y * np.log(y_hat + epsilon) + (1 - y) * np.log(1 - y_hat + epsilon)
    )

    return cost

In [204]:
def initialiser(n_l,n_n,x):
    
    n_n.insert(0,x.shape[0])
    weights = []
    biases=[]
    for i in range(n_l):
        weights.append(np.random.rand(n_n[i+1],n_n[i] )*0.01 ) # Add random 2x2 arrays
        biases.append(np.zeros((n_n[i+1],1)))
    return weights,biases

In [205]:
def model(x,y,n_l,n_n,act,epochs=10000,alpha=0.05):
    W,B=initialiser(n_l,n_n,x)
    m=x.shape[1]
    
    #forward pass
    for j in range (epochs):
        input_data=x
        A=[]
        Z=[]
        for i in range (n_l):
            z, a = forward(W[i], input_data, B[i], activation[act[i]])
            Z.append(z)
            A.append(a)
            input_data=A[i]
        # backward pass
        dA=[]
        dZ=[]
        dW=[]
        dB=[]
        da_o= - (np.divide(y, A[n_l-1]) - np.divide(1 - y, 1 - A[n_l-1]))
        dA.append(da_o)
        for i in range (n_l):     
            dz = dA[i] * derivatives[act[n_l - 1 - i]](Z[n_l - 1 - i])
            dZ.append(dz)
            prev_A = A[(n_l - 2 - i)] if (n_l - 2 - i) >= 0 else x
            dw = (1/m) * np.dot(dZ[i], prev_A.T)
            dW.append(dw)
            db = (1/m) * np.sum(dZ[i], axis=1, keepdims=True)
            dB.append(db)
            da = np.dot(W[n_l-1-i].T , dZ[i])
            dA.append(da)
        #updating params
        for i in range(n_l):
            W[i]=W[i]-alpha*dW[n_l-1-i]
            B[i]=B[i]-alpha*dB[n_l-1-i]
            
        if (j%100==0):
            cost=compute_cost(y,A[n_l-1])
            print(f"cost at iteration {j}:{cost}")
    return W,B,A

In [210]:
l=3
neurons=[8,6,1]
act=[0,0,1]


In [207]:
def predict(W, B, x, act):
    """
    Performs a forward pass using trained weights and returns the final activation (predicted probabilities).
    """
    input_data = x
    for i in range(len(W)):
        z = np.dot(W[i], input_data) + B[i]
        input_data = activation[act[i]](z)
    return input_data  # final layer output (probabilities)


In [208]:
def accuracy(y_true, y_pred):
    """
    Computes accuracy between true labels and predicted probabilities.
    """
    y_pred_labels = (y_pred > 0.5).astype(int)
    return np.mean(y_pred_labels == y_true)



In [211]:
# After training
W_trained, B_trained,a = model(x_train, y_train, l, neurons, act, epochs=10000, alpha=0.05)

# Get predictions
y_pred = predict(W_trained, B_trained, x_test, act)

# Accuracy
print("Accuracy:", accuracy(y_test, y_pred))

cost at iteration 0:11.54057708083656
cost at iteration 100:11.575866669403348
cost at iteration 200:11.575866669403348
cost at iteration 300:11.575866669403348
cost at iteration 400:11.575866669403348
cost at iteration 500:11.575866669403348
cost at iteration 600:11.575866669403348
cost at iteration 700:11.575866669403348
cost at iteration 800:11.575866669403348
cost at iteration 900:11.575866669403348
cost at iteration 1000:11.575866669403348
cost at iteration 1100:11.575866669403348
cost at iteration 1200:11.575866669403348
cost at iteration 1300:11.575866669403348
cost at iteration 1400:11.575866669403348
cost at iteration 1500:11.575866669403348
cost at iteration 1600:11.575866669403348
cost at iteration 1700:11.575866669403348
cost at iteration 1800:11.575866669403348
cost at iteration 1900:11.575866669403348
cost at iteration 2000:11.575866669403348
cost at iteration 2100:11.575866669403348
cost at iteration 2200:11.575866669403348
cost at iteration 2300:11.575866669403348
cost 